In [1]:
import os
os.chdir("../")
%pwd

'd:\\PracticeProjects\\NL_Text_Summarization\\NL_TEXT_SUMMARIZER'

In [2]:
# This block will later be moved to entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [3]:
# Define Config Manager
from NLPTextSummarizer.constants import *
from NLPTextSummarizer.utils.common import read_yaml, create_directories

# 4. Update the configuration manager(./src/project_name/config/configuration.py)
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        #print(CONFIG_FILE_PATH)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 
        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        # Move config path dict from config.yaml
        config = self.config.data_transformation
        # create the directories as mentioned in config.yaml
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        return data_transformation_config

In [8]:
# Define components
# 5. Update the components (./src/project_name/components/__init__.py)
import os
from NLPTextSummarizer.logging import logger
from transformers import AutoTokenizer # type: ignore
from datasets import load_from_disk # type: ignore

class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch["dialogue"], max_length = 1024, truncation = True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch["summary"], max_length = 128, truncation = True)

        return {
            "input_ids" : input_encodings["input_ids"],
            "attention_mask" : input_encodings["attention_mask"],
            "labels" : target_encodings["input_ids"]
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [10]:
# Pipeline component
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config) 
    data_transformation.convert()
except Exception as e:
    raise e

[2024-05-06 23:33:35,637: INFO: common: yaml file: D:\PracticeProjects\NL_Text_Summarization\NL_TEXT_SUMMARIZER\config\config.yaml loaded successfully]
[2024-05-06 23:33:35,638: INFO: common: yaml file: D:\PracticeProjects\NL_Text_Summarization\NL_TEXT_SUMMARIZER\params.yaml loaded successfully]
[2024-05-06 23:33:35,639: INFO: common: created directory at: artifacts]
[2024-05-06 23:33:35,640: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\Softwares\Anaconda3\envs\NLP_TextSummarization_VENV\Lib\site-packages\transformers\tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 163168.34 examples/s]
